# Cherry pick compounds
- Given a random assortment of compounds in our library, find plate locations and reduce the number of plates needed to pick from

In [ ]:
import sys
sys.path.append('/home/apaulson/repos/data-proc')
import plate_maps as pm

import pandas as pd
import os
import sys
import numpy as np
pd.set_option('display.max_columns', 90)
from pathlib import Path, PureWindowsPath

import importlib
%config Completer.use_jedi=False
print(pd.__version__)

cnx = pm.create_mysql_cnx()

# Look up drug info from HiTS db

In [29]:
df=pd.read_csv('/Volumes/Shared/SMDC/All_Compounds/ATOM_PARP/SPR planning with IDs.csv')
df.head(2)
df=df.rename(columns={'alias4_value':'SMDC_ID'})

In [30]:
ids=df[~df.SMDC_ID.isna()].SMDC_ID.astype(int).astype(str).tolist()
format_strings = ','.join(['%s'] * len(ids))

query="""
select * from INV_WELL inner join INV_PLATE on IW_IP_ID=IP_ID inner join INV_PLATE_TYPE on IP_IPT_ID=IPT_ID inner join INV_COMP_LOT on IW_ICL_ID=ICL_ID
where IW_ICL_ID in (
    select ICL_ID from INV_COMP_LOT where ICL_IC_ID in (%s) group by ICL_IC_ID) 
and IW_IP_ID in (
    select IP_ID from INV_PLATE where IP_IPT_ID not in (3,8,18));
"""% format_strings
# and IW_SOURCE_ID is NULL;
# """
fullresults=pd.read_sql(query, cnx, params=tuple(ids))

cnx.dispose()

# Refine results from query

In [31]:
results=fullresults.copy(deep=True)
print(results.shape)
results.head()

(3753, 69)


,IW_ID,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES,IP_ID,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER,IPT_ID,IPT_SDESC,IPT_LDESC,IPT_NOTES,IPT_STATUS_ID,IPT_INS_BY,IPT_INS_DATE,ICL_ID,ICL_IC_ID,ICL_LOT_NUM,ICL_IV_ID,ICL_ILT_ID,ICL_ALIAS,ICL_PARENT_IC_ID,ICL_IAT_ID,ICL_ICO_ID,ICL_VEN_COLLECTION,ICL_SALT,ICL_SALT_NUM,ICL_SOLVATE,ICL_SOLVATE_NUM,ICL_DATE_PREPARED,ICL_APPEARANCE,ICL_AMT_PREPARED,ICL_STOCK_CONC_UM,ICL_AMT_UNITS,ICL_PURITY,ICL_SOLUBILITY,ICL_IN_ID,ICL_PAGE_NUM,ICL_COMMENTS,ICL_HP_ID,ICL_STATUS_ID,ICL_INS_BY,ICL_INS_DATE,class,ICL_SCREENING_CENTER
0,372972,NaN,3886,None,12,A12,94070,0.0,10000.0,NaN,None,1.0,2.0,2006-09-05 14:22:49,0.0,None,3886,NaN,1270CHDV,1,1,None,1270CHDV-C-MAP001,2,3.0,1270CHDV-C-MAP001,62,ChemDiv_Verkman50K,Returned to Verkman,2,2006-09-05 14:19:25,SMDC,1,MA,Master Plate,None,1,2,2006-07-25 13:46:27,94070,88704,1,3.0,2,4460-0006,None,Vendor_ID,NaN,None,None,0.0,None,0.0,None,None,NaN,NaN,None,NaN,None,None,None,None,None,1,2,2006-09-05 14:18:36,hits.lot.CompoundLotVendor,SMDC
1,477165,NaN,4632,None,45,B21,94070,59.0,1000.0,NaN,None,2.0,2.0,2006-09-05 14:23:46,0.0,None,4632,NaN,01554CD,2,2,None,01554cd-dt070-sp001,2,3.0,None,62,ChemDiv_Verkman50K,None,2,2006-09-05 14:19:25,SMDC,2,SP,Screening Plate,None,1,2,2006-07-25 13:46:27,94070,88704,1,3.0,2,4460-0006,None,Vendor_ID,NaN,None,None,0.0,None,0.0,None,None,NaN,NaN,None,NaN,None,None,None,None,None,1,2,2006-09-05 14:18:36,hits.lot.CompoundLotVendor,SMDC
2,427078,NaN,4449,None,70,F10,94071,147.0,10000.0,NaN,None,2.0,2.0,2006-09-05 14:23:19,0.0,None,4449,NaN,01833MS,1,1,None,01833ms-ma01,1,5.0,199514,103,Microsource_bioactives,None,2,2006-09-05 14:19:25,SMDC,1,MA,Master Plate,None,1,2,2006-07-25 13:46:27,94071,88704,2,5.0,2,01500568,None,Vendor_ID,103.0,Microsource_bioactives,None,0.0,None,0.0,None,None,NaN,NaN,None,NaN,None,None,None,None,None,1,2,2006-09-05 14:18:53,hits.lot.CompoundLotVendor,SMDC
3,619076,NaN,5001,None,260,K20,94071,41.8,10000.0,NaN,None,6.0,2.0,2006-09-05 14:25:04,0.0,None,5001,NaN,01870MS,4,2,None,01870ms-in01-01832,1,5.0,None,103,Microsource_bioactives,None,2,2006-09-05 14:19:25,SMDC,4,IN,Intermediate (replica) Plate,None,1,2,2006-07-25 13:46:27,94071,88704,2,5.0,2,01500568,None,Vendor_ID,103.0,Microsource_bioactives,None,0.0,None,0.0,None,None,NaN,NaN,None,NaN,None,None,None,None,None,1,2,2006-09-05 14:18:53,hits.lot.CompoundLotVendor,SMDC
4,619460,NaN,5002,None,260,K20,94071,16.0,1000.0,NaN,None,8.0,2.0,2006-09-05 14:25:04,0.0,None,5002,NaN,01870MS,2,2,None,01870ms-sp01-01832,0,5.0,None,103,Microsource_bioactives,None,2,2006-09-05 14:19:25,SMDC,2,SP,Screening Plate,None,1,2,2006-07-25 13:46:27,94071,88704,2,5.0,2,01500568,None,Vendor_ID,103.0,Microsource_bioactives,None,0.0,None,0.0,None,None,NaN,NaN,None,NaN,None,None,None,None,None,1,2,2006-09-05 14:18:53,hits.lot.CompoundLotVendor,SMDC


In [32]:
# look at number of unique compounds and where they exist in plates
print(results.ICL_IC_ID.nunique(), results.IP_BARCODE.nunique())
results=results[~results.IP_BARCODE.str.contains('-Q1')]
results=results[~results.IP_BARCODE.str.contains('-Q2')]
results=results[~results.IP_BARCODE.str.contains('-Q3')]
results=results[~results.IP_BARCODE.str.contains('-Q4')]
print(results.ICL_IC_ID.nunique(), results.IP_BARCODE.nunique())

100 630
100 431


In [33]:
# reduce plates to gridnums to find master plate for each set
gns = results[['ICL_IC_ID','IP_GRID_NUM', 'IP_IPT_ID']]
ngns=gns.ICL_IC_ID.nunique()
gns = gns.drop_duplicates()
gnvals = gns.IP_GRID_NUM.value_counts()
gnlist=pd.DataFrame(gnvals).sort_values(by='IP_GRID_NUM',ascending=True).index
for gn in gnlist:
    nicids = gns[gns.IP_GRID_NUM!=gn].ICL_IC_ID.nunique()
    if nicids == ngns:
        gns = gns[gns.IP_GRID_NUM!=gn]
print(gns.IP_GRID_NUM.nunique())

22


In [34]:
gns.ICL_IC_ID.nunique(),gns.IP_GRID_NUM.nunique()

(100, 22)

In [35]:
gns=gns.IP_GRID_NUM.value_counts().index

In [36]:
print(results.shape)
results=results[results.IP_GRID_NUM.isin(gns)]
print(results.shape)

(3154, 69)
(1029, 69)


In [37]:
results.head(2)

,IW_ID,IW_SOURCE_ID,IW_IP_ID,IW_BARCODE,IW_WELL_INDEX,IW_COORDINATES,IW_ICL_ID,IW_VOL_UL,IW_CONC_UM,IW_FINAL_CONC_NM,IW_DILUTION_FACTOR,IW_FREEZETHAW,IW_INS_BY,IW_INS_DATE,IW_CONTAM,IW_NOTES,IP_ID,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER,IPT_ID,IPT_SDESC,IPT_LDESC,IPT_NOTES,IPT_STATUS_ID,IPT_INS_BY,IPT_INS_DATE,ICL_ID,ICL_IC_ID,ICL_LOT_NUM,ICL_IV_ID,ICL_ILT_ID,ICL_ALIAS,ICL_PARENT_IC_ID,ICL_IAT_ID,ICL_ICO_ID,ICL_VEN_COLLECTION,ICL_SALT,ICL_SALT_NUM,ICL_SOLVATE,ICL_SOLVATE_NUM,ICL_DATE_PREPARED,ICL_APPEARANCE,ICL_AMT_PREPARED,ICL_STOCK_CONC_UM,ICL_AMT_UNITS,ICL_PURITY,ICL_SOLUBILITY,ICL_IN_ID,ICL_PAGE_NUM,ICL_COMMENTS,ICL_HP_ID,ICL_STATUS_ID,ICL_INS_BY,ICL_INS_DATE,class,ICL_SCREENING_CENTER
20,4479377,NaN,15377,None,117,E21,705263,80.0,10000.0,NaN,None,NaN,59.0,2011-06-06 17:59:49,0.0,None,15377,NaN,SP0003,6,2,None,SP0003-A,1,5.0,Spectrum 3,149,Spectrum,None,59,2011-06-06 17:59:49,CSC,6,PT,Pin-tool high concentration screening plate,None,1,59,2009-07-29,705263,88704,7,5.0,2,01500568,None,Vendor_ID,149.0,Spectrum,None,0.0,None,NaN,None,None,NaN,NaN,None,NaN,None,None,None,None,None,1,59,2011-06-06 17:59:12,hits.lot.CompoundLotVendor,CSC
21,6263095,NaN,20344,None,117,E21,705263,80.0,10000.0,NaN,None,NaN,107.0,2013-10-31 08:59:13,0.0,None,20344,15377.0,SP0003,6,2,None,SP0003-B,1,5.0,Spectrum 3,149,Spectrum,,107,2013-10-31 08:54:29,CSC,6,PT,Pin-tool high concentration screening plate,None,1,59,2009-07-29,705263,88704,7,5.0,2,01500568,None,Vendor_ID,149.0,Spectrum,None,0.0,None,NaN,None,None,NaN,NaN,None,NaN,None,None,None,None,None,1,59,2011-06-06 17:59:12,hits.lot.CompoundLotVendor,CSC


In [38]:
results.columns

Index(['IW_ID', 'IW_SOURCE_ID', 'IW_IP_ID', 'IW_BARCODE', 'IW_WELL_INDEX',
       'IW_COORDINATES', 'IW_ICL_ID', 'IW_VOL_UL', 'IW_CONC_UM',
       'IW_FINAL_CONC_NM', 'IW_DILUTION_FACTOR', 'IW_FREEZETHAW', 'IW_INS_BY',
       'IW_INS_DATE', 'IW_CONTAM', 'IW_NOTES', 'IP_ID', 'IP_SOURCE_ID',
       'IP_GRID_NUM', 'IP_IPT_ID', 'IP_IPF_ID', 'IP_DESC', 'IP_BARCODE',
       'IP_STATUS_ID', 'IP_IV_ID', 'IP_VEN_PLID', 'IP_ICO_ID', 'IP_COLLECTION',
       'IP_NOTES', 'IP_INS_BY', 'IP_INS_DATE', 'IP_SCREENING_CENTER', 'IPT_ID',
       'IPT_SDESC', 'IPT_LDESC', 'IPT_NOTES', 'IPT_STATUS_ID', 'IPT_INS_BY',
       'IPT_INS_DATE', 'ICL_ID', 'ICL_IC_ID', 'ICL_LOT_NUM', 'ICL_IV_ID',
       'ICL_ILT_ID', 'ICL_ALIAS', 'ICL_PARENT_IC_ID', 'ICL_IAT_ID',
       'ICL_ICO_ID', 'ICL_VEN_COLLECTION', 'ICL_SALT', 'ICL_SALT_NUM',
       'ICL_SOLVATE', 'ICL_SOLVATE_NUM', 'ICL_DATE_PREPARED', 'ICL_APPEARANCE',
       'ICL_AMT_PREPARED', 'ICL_STOCK_CONC_UM', 'ICL_AMT_UNITS', 'ICL_PURITY',
       'ICL_SOLUBILITY', 'I

In [39]:
results=results[['IP_COLLECTION','ICL_VEN_COLLECTION','ICL_IC_ID', 'IP_GRID_NUM','IP_BARCODE',
       'IPT_LDESC','IP_DESC', 'IW_COORDINATES', 'IW_VOL_UL', 'IW_CONC_UM',
       'IW_FINAL_CONC_NM', 'IW_DILUTION_FACTOR', 'IW_FREEZETHAW',
       'IW_ICL_ID',  'IP_ID', 'IP_IPT_ID', 'IP_IPF_ID',
       'IP_ICO_ID', 'IP_NOTES', 'ICL_ID','ICL_ALIAS',
       'ICL_ICO_ID', 'ICL_STOCK_CONC_UM', 'ICL_AMT_UNITS', 'ICL_PURITY', 'ICL_SOLUBILITY',
       'ICL_IN_ID', 'ICL_PAGE_NUM', 'ICL_COMMENTS',]]
print(results.shape)
results=results.dropna(how='all', axis=1)
print(results.shape)
results.head()

(1029, 29)
(1029, 24)


,IP_COLLECTION,ICL_VEN_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IP_BARCODE,IPT_LDESC,IP_DESC,IW_COORDINATES,IW_VOL_UL,IW_CONC_UM,IW_FREEZETHAW,IW_ICL_ID,IP_ID,IP_IPT_ID,IP_IPF_ID,IP_ICO_ID,IP_NOTES,ICL_ID,ICL_ALIAS,ICL_ICO_ID,ICL_AMT_UNITS,ICL_PURITY,ICL_SOLUBILITY,ICL_COMMENTS
20,Spectrum,Spectrum,88704,SP0003,SP0003-A,Pin-tool high concentration screening plate,None,E21,80.0,10000.0,NaN,705263,15377,6,2,149,None,705263,01500568,149.0,None,NaN,None,None
21,Spectrum,Spectrum,88704,SP0003,SP0003-B,Pin-tool high concentration screening plate,None,E21,80.0,10000.0,NaN,705263,20344,6,2,149,,705263,01500568,149.0,None,NaN,None,None
22,Spectrum,Spectrum,88704,SP0003,SP0003-C,Pin-tool high concentration screening plate,None,E21,80.0,10000.0,NaN,705263,20345,6,2,149,,705263,01500568,149.0,None,NaN,None,None
23,Spectrum,Spectrum,88704,SP0003,SP0003-D,Pin-tool high concentration screening plate,None,E21,80.0,10000.0,NaN,705263,20346,6,2,149,,705263,01500568,149.0,None,NaN,None,None
51,SMDC Drugs,FDA Supplemental Purchase 2013-1,88704,05041PK,05041PK-96Q2,Library plates contained solely within the Biomek,Original Quad,H02,1055.0,10000.0,1.0,1047779,21256,7,1,171,Originally added into the Pharamkon1600 set,1047779,4499-40-5,72.0,mg,80.0,None,"Original source: Kemprotec Limited, Barcode: B..."


In [40]:
# only keep public collections
results = results[~results.IP_COLLECTION.isin(
    ['NCI', 'Iconix_FDA','NCI Bioactive',
     'ChemDiv_Verkman50K', 'Enzo Dilution', 
     'Microsource_bioactives', 'LifeChem_5000_ProtInhib', 'ATOM', 'Cottam_Clones','Cottam_Clones_Pilot'])]

In [41]:
results.shape

(991, 24)

In [42]:
results_mini=results[['IP_COLLECTION','ICL_IC_ID','IP_GRID_NUM','IW_COORDINATES']]

In [43]:
results_mini=results_mini.drop_duplicates()
results_mini

,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES
20,Spectrum,88704,SP0003,E21
51,SMDC Drugs,88704,05041PK,H02
52,SMDC Drugs,88704,05041PK,O04
66,SMDC Drugs,88704,05041PK,F09
67,SMDC Drugs,88704,05041PK,K17
...,...,...,...,...
3733,TargetMol,1076574,05304TM,I20
3737,TargetMol,1076804,05307TM,H08
3741,TargetMol,1076811,05307TM,I14
3745,TargetMol,1076833,05307TM,M04


In [44]:
# sort data so that the number of compounds per plate is maximized
sorter=results_mini.IP_GRID_NUM.value_counts().index

results_mini.IP_GRID_NUM = results_mini.IP_GRID_NUM.astype("category")
results_mini.IP_GRID_NUM = results_mini.IP_GRID_NUM.cat.set_categories(sorter)

In [45]:
results_mini.sort_values('IP_GRID_NUM')

,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES
2419,SMDC Drugs,254926,05040PK,H12
2209,SMDC Drugs,131739,05040PK,G17
2536,SMDC Drugs,255751,05040PK,A03
2617,SMDC Drugs,255870,05040PK,K09
2659,SMDC Drugs,255931,05040PK,E08
...,...,...,...,...
3648,SelleckBioactives,972704,L1700-16,B09
229,SelleckBioactives,129988,L1700-16,B07
3526,NCI Protease,811673,ML00107005,M06
3671,Pitt ATG4B Electrophiles,973368,UPCDCD0350005,F15


In [47]:
# drop duplicate compounds while keeping the ones in the most populous plate
results_mini_new = pd.DataFrame([], columns=results_mini.columns)
for gn in results_mini.IP_GRID_NUM:
    icids=results_mini_new.ICL_IC_ID.unique()
    results_mini_top=results_mini[results_mini.IP_GRID_NUM==gn]
    results_mini_top=results_mini_top[~results_mini_top.ICL_IC_ID.isin(icids)]
    results_mini_new=pd.concat([results_mini_new,results_mini_top])

In [48]:
# drop duplicate compounds while keeping the ones in the most populous plate
sorter=results_mini_new.IP_GRID_NUM.value_counts().index

results_mini_new.IP_GRID_NUM = results_mini_new.IP_GRID_NUM.astype("category")
results_mini_new.IP_GRID_NUM = results_mini_new.IP_GRID_NUM.cat.set_categories(sorter)
results_mini_new = results_mini_new.sort_values('IP_GRID_NUM')
results_mini_new = results_mini_new[~results_mini_new.ICL_IC_ID.duplicated(keep='first')]
results_mini_new.head()

,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES
3347,SMDC Drugs,285985,05040PK,D18
3316,SMDC Drugs,285982,05040PK,D12
3285,SMDC Drugs,285979,05040PK,D06
2725,SMDC Drugs,255968,05040PK,F03
2617,SMDC Drugs,255870,05040PK,K09


In [49]:
compounds_per_plate=pd.DataFrame(results_mini_new.IP_GRID_NUM.value_counts()).reset_index()
compounds_per_plate.columns=['IP_GRID_NUM', 'num_cmpds']
collection_map=dict(zip(results_mini_new.IP_GRID_NUM, results_mini_new.IP_COLLECTION))
compounds_per_plate['Collection']=compounds_per_plate.IP_GRID_NUM.map(collection_map)
compounds_per_plate=compounds_per_plate[['Collection','IP_GRID_NUM','num_cmpds']]

In [50]:
compounds_per_plate.to_csv('SPR_compounds_per_plate.csv', index=False)
results_mini_new.to_csv('SPR_locations.csv', index=False)

In [51]:
display(results_mini_new.head(2))
display(df.head(2))

,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES
3347,SMDC Drugs,285985,05040PK,D18
3316,SMDC Drugs,285982,05040PK,D12


,Name,MW,human CL (mL/min/kg),fraction unbound in plasma (fu) exp,LogD QSAR,MokapKa_a_pred2[1],MokapKa_a_pred2[2],MoKapKa_b_pred2[1],MoKapKa_b_pred2[2],human VDss (L/kg),B2Pexp,SMILES,alias4,SMDC_ID
0,Actisomide,370,6.79,0.6954,-0.34,14.0,14,8.6,0,0.58,0.78,CC1=NC(=O)[C@]([C@@H]2N1CCCC2)(CCN(C(C)C)C(C)C...,SMDC_ID,NaN
1,Acyclovir,225,4.70,0.8500,-1.52,9.0,14,4.1,0,0.71,1.04,C1=NC2=C(N1COCCO)N=C(NC2=O)N,SMDC_ID,131362.0


In [52]:
df=df.merge(results_mini_new, how='left', left_on='SMDC_ID',right_on='ICL_IC_ID')

In [53]:
results_mini_new.ICL_IC_ID.nunique()

99

In [54]:
results_mini_new.head()

,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES
3347,SMDC Drugs,285985,05040PK,D18
3316,SMDC Drugs,285982,05040PK,D12
3285,SMDC Drugs,285979,05040PK,D06
2725,SMDC Drugs,255968,05040PK,F03
2617,SMDC Drugs,255870,05040PK,K09


In [55]:
results_mini_new.IP_GRID_NUM.value_counts()

IP_GRID_NUM
05302TM          15
SP0005           13
05303TM          12
05307TM           8
05040PK           7
05301TM           7
SP0001            7
05041PK           5
05304TM           5
05038PK           4
SP0003            3
05321TM           3
05317TM           2
SP0006            1
L1700-04          1
05322TM           1
05315TM           1
L1700-16          1
ML00107005        1
UPCDCD0350005     1
05312TM           1
Name: count, dtype: int64

In [56]:
print(df.shape)
df=df[~df.IW_COORDINATES.isna()]
print(df.shape)
df=df.drop_duplicates()
print(df.shape)
df.head()

(131, 18)
(110, 18)
(99, 18)


,Name,MW,human CL (mL/min/kg),fraction unbound in plasma (fu) exp,LogD QSAR,MokapKa_a_pred2[1],MokapKa_a_pred2[2],MoKapKa_b_pred2[1],MoKapKa_b_pred2[2],human VDss (L/kg),B2Pexp,SMILES,alias4,SMDC_ID,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES
1,Acyclovir,225,4.70,0.85,-1.52,9.0,14,4.1,0,0.71,1.04,C1=NC2=C(N1COCCO)N=C(NC2=O)N,SMDC_ID,131362.0,TargetMol,131362,05304TM,J22
2,Almotriptan,335,8.90,0.60,-0.11,11.0,14,8.7,0,2.20,0.86,CN(C)CCC1=CNC2=C1C=C(C=C2)CS(=O)(=O)N3CCCC3,SMDC_ID,131687.0,Pharmakon1600,131687,05038PK,M08
3,Amiodarone,645,1.90,0.05,6.10,14.0,14,8.7,0,66.00,0.73,CCCCC1=C(C2=CC=CC=C2O1)C(=O)C3=CC(=C(C(=C3)I)O...,SMDC_ID,130213.0,TargetMol,130213,05303TM,H05
4,Amphotericin B,924,0.17,0.03,0.19,4.6,14,8.6,0,0.62,0.87,C[C@H]1/C=C/C=C/C=C/C=C/C=C/C=C/C=C/[C@@H](C[C...,SMDC_ID,285833.0,TargetMol,285833,05302TM,J18
7,Antipyrine,188,0.64,0.93,0.43,14.0,14,0.0,0,0.77,1.00,CC1=CC(=O)N(N1C)C2=CC=CC=C2,SMDC_ID,129988.0,Spectrum,129988,SP0005,M17


In [57]:
df=df.drop_duplicates()
print(df.shape)

(99, 18)


In [58]:
df.to_csv('SPR_compound_list_large.csv', index=False)

In [59]:
df.sort_values('SMDC_ID')

,Name,MW,human CL (mL/min/kg),fraction unbound in plasma (fu) exp,LogD QSAR,MokapKa_a_pred2[1],MokapKa_a_pred2[2],MoKapKa_b_pred2[1],MoKapKa_b_pred2[2],human VDss (L/kg),B2Pexp,SMILES,alias4,SMDC_ID,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES
119,Theophylline,180,0.86,0.61,0.12,8.8,14,1.6,0,0.51,1.02,CN1C2=C(C(=O)N(C1=O)C)NC=N2,SMDC_ID,88704.0,Spectrum,88704,SP0003,E21
49,Erythromycin,734,5.60,0.10,1.03,13.0,14,8.8,0,0.95,0.91,CC[C@@H]1[C@@]([C@@H]([C@H](C(=O)[C@@H](C[C@@]...,SMDC_ID,129937.0,TargetMol,129937,05302TM,P14
7,Antipyrine,188,0.64,0.93,0.43,14.0,14,0.0,0,0.77,1.00,CC1=CC(=O)N(N1C)C2=CC=CC=C2,SMDC_ID,129988.0,Spectrum,129988,SP0005,M17
18,Cefazolin,455,0.89,0.18,-1.82,2.3,14,1.6,0,0.12,0.64,CC1=NN=C(S1)SCC2=C(N3[C@@H]([C@@H](C3=O)NC(=O)...,SMDC_ID,130011.0,Pharmakon1600,130011,05038PK,B21
30,Chlorpromazine,319,16.00,0.12,3.17,14.0,14,9.0,0,11.00,1.20,CN(C)CCCN1C2=CC=CC=C2SC3=C1C=C(C=C3)Cl,SMDC_ID,130028.0,Spectrum,130028,SP0001,F10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,Cimetidine,252,8.10,0.75,0.68,14.0,14,6.4,0,1.20,0.97,CC1=C(N=CN1)CSCCNC(=NC)NC#N,SMDC_ID,1076574.0,TargetMol,1076574,05304TM,I20
65,Imipenem,299,3.00,0.86,-1.65,2.9,14,9.9,0,0.24,1.14,C[C@H]([C@@H]1[C@H]2CC(=C(N2C1=O)C(=O)O)SCCN=CN)O,SMDC_ID,1076804.0,TargetMol,1076804,05307TM,H08
35,Citalopram,324,4.30,0.20,1.48,14.0,14,9.0,0,12.00,1.30,CN(C)CCCC1(C2=C(CO1)C=C(C=C2)C#N)C3=CC=C(C=C3)F,SMDC_ID,1076811.0,TargetMol,1076811,05307TM,I14
53,Fleroxacin,369,2.40,0.73,-0.54,5.5,14,8.0,0,1.60,0.69,CN1CCN(CC1)C2=C(C=C3C(=C2F)N(C=C(C3=O)C(=O)O)C...,SMDC_ID,1076833.0,TargetMol,1076833,05307TM,M04


In [60]:
df.SMDC_ID.nunique()

99

In [61]:
df.columns

Index(['Name', 'MW', 'human CL (mL/min/kg)',
       'fraction unbound in plasma (fu) exp', 'LogD QSAR',
       'MokapKa_a_pred2[1]', 'MokapKa_a_pred2[2]', 'MoKapKa_b_pred2[1]',
       'MoKapKa_b_pred2[2]', 'human VDss (L/kg)', 'B2Pexp', 'SMILES', 'alias4',
       'SMDC_ID', 'IP_COLLECTION', 'ICL_IC_ID', 'IP_GRID_NUM',
       'IW_COORDINATES'],
      dtype='object')

# draw plate maps

In [63]:
import string
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})

In [64]:
results_mini_new['row'] = results_mini_new.IW_COORDINATES.str[0:1]
results_mini_new['col'] = results_mini_new.IW_COORDINATES.str[1:]
results_mini_new.col = results_mini_new.col.astype(int)
results_mini_new.head()

,IP_COLLECTION,ICL_IC_ID,IP_GRID_NUM,IW_COORDINATES,row,col
3347,SMDC Drugs,285985,05040PK,D18,D,18
3316,SMDC Drugs,285982,05040PK,D12,D,12
3285,SMDC Drugs,285979,05040PK,D06,D,6
2725,SMDC Drugs,255968,05040PK,F03,F,3
2617,SMDC Drugs,255870,05040PK,K09,K,9


In [65]:
results_mini_new=results_mini_new.sort_values(['IP_COLLECTION','IP_GRID_NUM'])

In [66]:
pdf = matplotlib.backends.backend_pdf.PdfPages("DEG_compound_plate_maps.pdf")

for gridnum in results_mini_new.IP_GRID_NUM.unique():
    plate = pd.DataFrame(index = list(string.ascii_uppercase[0:16]), columns = np.arange(1,25))
    plate = plate.fillna(1)
    plate2= pd.DataFrame(index = list(string.ascii_uppercase[0:16]), columns = np.arange(1,25))
    plate2 = plate2.fillna('')

    test = results_mini_new[results_mini_new.IP_GRID_NUM==gridnum]

    for i, row in test.iterrows():
        plate.loc[row.row, row.col]=0
        plate2.loc[row.row,row.col]=row.IW_COORDINATES

    fig, ax = plt.subplots(1, figsize=(12,8))
    sns.heatmap(plate,ax = ax, robust=False, square=True,linewidths=0.5, cbar=False, annot=plate2, fmt=':^', vmin=-5, vmax=1)
    plt.yticks(rotation=0);
    plt.title(f'{row.IP_COLLECTION} {row.IP_GRID_NUM}');
    pdf.savefig( fig )
    plt.close()
pdf.close()   